# Tempo Difference Explorer

Align a performance with its reference score, inspect per-note time differences with
LOESS smoothing, and interactively explore a scrollable piano-roll. A simple sine-wave
synthesizer lets you audition the performed notes directly in the notebook.


In [1]:
from __future__ import annotations

import sys
from pathlib import Path

import numpy as np
from statsmodels.nonparametric.smoothers_lowess import lowess

from bokeh.io import output_notebook, show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Range1d
from bokeh.plotting import figure

from IPython.display import Audio

NOTEBOOK_DIR = Path.cwd().resolve()
BACKEND_ROOT = NOTEBOOK_DIR.parents[2]
SRC_ROOT = BACKEND_ROOT / "src"
if str(SRC_ROOT) not in sys.path:
    sys.path.insert(0, str(SRC_ROOT))

from scoring import extract_midi_notes, extract_pb_notes
from scoring.edit_distance import find_ops
from scoring.notes_pb2 import NoteList

output_notebook()

/Users/timothyliu/PycharmProjects/note/backend/.venv/lib/python3.13/site-packages/pretty_midi/instrument.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Loading BokehJS ...

In [2]:
# Configure the reference and performance sources.
# Paths can be absolute or relative to this notebook directory.
ACTUAL_PATH = Path("../scores/spider dance.scoredata")
PLAYED_PATH = Path("../audio/spider dance played.midi")


def resolve_path(path: Path) -> Path:
    return path if path.is_absolute() else (NOTEBOOK_DIR / path).resolve()


ACTUAL_PATH = resolve_path(ACTUAL_PATH)
PLAYED_PATH = resolve_path(PLAYED_PATH)
ACTUAL_PATH, PLAYED_PATH

(PosixPath('/Users/timothyliu/PycharmProjects/note/backend/resources/scores/spider dance.scoredata'),
 PosixPath('/Users/timothyliu/PycharmProjects/note/backend/resources/audio/spider dance played.midi'))

In [3]:
def load_note_list(path: Path) -> NoteList:
    suffix = path.suffix.lower()
    if suffix in {".scoredata", ".pb"}:
        return extract_pb_notes(path.read_bytes())
    if suffix in {".midi", ".mid"}:
        return extract_midi_notes(str(path))
    raise ValueError(f"Unsupported file format: {path}")


actual_notes = load_note_list(ACTUAL_PATH)
played_notes = load_note_list(PLAYED_PATH)
len(actual_notes.notes), len(played_notes.notes)

(1774, 1699)

In [4]:
# Align the performance to the reference score.
_, aligned_pairs = find_ops(actual_notes.notes, played_notes.notes)
aligned_pairs = sorted((int(a), int(b)) for a, b in aligned_pairs)
len(aligned_pairs)

2025-09-28 14:50:53.020 | INFO     | scoring.edit_distance:find_ops:139 - 	[preprocess] took 2.472 ms
2025-09-28 14:50:54.074 | INFO     | scoring.edit_distance:find_ops:140 - 	[edit_distance] took 1053.756 ms
2025-09-28 14:50:54.077 | INFO     | scoring.edit_distance:find_ops:142 - 	[postprocess] took 1.921 ms


1563

In [11]:
# Build aligned time arrays and compute differences (reference minus performed).
actual_times = np.array(
    [float(actual_notes.notes[a].start_time) for a, _ in aligned_pairs]
)
played_times = np.array(
    [float(played_notes.notes[b].start_time) for _, b in aligned_pairs]
)
time_diffs = actual_times - played_times
note_indices = np.array([a for a, _ in enumerate(aligned_pairs)])
note_indices[:15], time_diffs[:5]

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([3.15166956, 3.14860214, 3.16308564, 3.16305515, 3.1497499 ]))

In [6]:
# Apply LOESS smoothing (0 < frac <= 1 controls window width).
LOESS_FRACTION = 0.1
loess_result = lowess(
    time_diffs, note_indices, frac=LOESS_FRACTION, return_sorted=False
)
loess_result[:5]

array([3.16596595, 3.17022263, 3.1744782 , 3.17873269, 3.18298613])

In [7]:
def build_roll_source(
    notes: NoteList, note_ids: np.ndarray, deltas: np.ndarray
) -> ColumnDataSource:
    left, right, aligned, bottom, top, labels, diffs = [], [], [], [], [], [], []
    for idx, note_idx in enumerate(note_ids):
        note = notes.notes[int(note_idx)]
        index_left = float(note_idx)
        index_right = index_left + 1.0

        pitch = int(note.pitch)
        left.append(index_left)
        right.append(index_right)
        aligned.append(idx)
        bottom.append(pitch - 0.45)
        top.append(pitch + 0.45)
        labels.append(int(note_idx))
        diffs.append(float(deltas[idx]))
    return ColumnDataSource(
        {
            "left": left,
            "right": right,
            "aligned": aligned,
            "bottom": bottom,
            "top": top,
            "note_index": labels,
            "delta": diffs,
        }
    )

In [8]:
ref_source = build_roll_source(actual_notes, note_indices, time_diffs)
perf_indices = np.array([b for _, b in aligned_pairs])
perf_source = build_roll_source(played_notes, perf_indices, -time_diffs)
ref_source.data.keys(), perf_source.data.keys()

(dict_keys(['left', 'right', 'aligned', 'bottom', 'top', 'note_index', 'delta']),
 dict_keys(['left', 'right', 'aligned', 'bottom', 'top', 'note_index', 'delta']))

In [9]:
# Piano-roll keyed to reference note index (default window ~10 notes).
all_left = ref_source.data["left"] + perf_source.data["left"]
all_right = ref_source.data["right"] + perf_source.data["right"]
all_bottom = ref_source.data["bottom"] + perf_source.data["bottom"]
all_top = ref_source.data["top"] + perf_source.data["top"]
if all_left:
    min_idx = float(min(all_left))
    max_idx = float(max(all_right))
else:
    min_idx, max_idx = 0.0, 10.0
roll_fig = figure(
    title="Scrollable Piano Roll",
    height=320,
    width=900,
    tools="xpan,xwheel_zoom,reset,hover",
    active_scroll="xwheel_zoom",
    active_drag="xpan",
)
roll_fig.grid.visible = False
roll_fig.yaxis.axis_label = "Pitch (MIDI)"
roll_fig.xaxis.axis_label = "Reference note index"

roll_fig.quad(
    source=ref_source,
    left="left",
    right="right",
    top="top",
    bottom="bottom",
    color="#1f77b4",
    alpha=0.6,
    legend_label="Reference",
)
roll_fig.quad(
    source=perf_source,
    left="left",
    right="right",
    top="top",
    bottom="bottom",
    color="#ff7f0e",
    alpha=0.6,
    legend_label="Performance",
)
roll_fig.hover.tooltips = [
    ("Note", "@note_index"),
    ("Δt (s)", "@delta{0.0000}"),
    ("Index span", "@left{0} - @right{0}"),
    ("Aligned", "@aligned"),
]
roll_fig.legend.click_policy = "hide"
roll_fig

figure(id='p1014', ...)

In [10]:
# Time-difference trace with LOESS smoothing.
diff_source = ColumnDataSource(
    {
        "note_idx": note_indices,
        "diff": time_diffs,
        "loess": loess_result,
    }
)
diff_fig = figure(
    title="Tempo Difference (Reference - Performance)",
    x_axis_label="Reference note index",
    y_axis_label="Time difference (s)",
    height=320,
    width=900,
    tools="xpan,xwheel_zoom,reset,hover",
    active_drag="xpan",
    active_scroll="xwheel_zoom",
)
diff_fig.circle(
    "note_idx", "diff", size=4, alpha=0.5, source=diff_source, legend_label="Raw"
)
diff_fig.line(
    "note_idx",
    "loess",
    line_width=3,
    color="firebrick",
    source=diff_source,
    legend_label="LOESS",
)
diff_fig.hover.tooltips = [
    ("Note", "@note_idx"),
    ("Δt (s)", "@diff{0.0000}"),
    ("LOESS", "@loess{0.0000}"),
]
diff_fig.legend.click_policy = "hide"
show(column(roll_fig, diff_fig))